# How to use Langchain for Geospatial Analysis for datasets in the Singapore context



# Introduction

In this notebook, we illustrate the use of Langchain for geospatial analysis for datasets in the Singaporean context. We will be using the following dataset, namely Angel Hsu's Chicken Rice dataset, which contains the geospatial coordinates of chicken rice stalls in Singapore, along with their food ratings, costs, and E-coli levels.

This dataset (as of 21 Jun 2023) is available at [this url](https://docs.google.com/spreadsheets/d/1hPis5_b_pwBpReao6QDx3wZK9_ExdohFLfOHI9ZWClM/edit#gid=2059674870), also posted on her [Youtube video](https://www.youtube.com/watch?v=toFZJ9ICWWY) titled 'I ate at 100 chicken rice stalls, and sent them to the lab... | Top 4 Chicken Rice in Singapore'

In [ ]:
# Installing langchain and openai libraries
# !pip install langchain openai

In [8]:
# Importing libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from langchain.agents import create_pandas_dataframe_agent
from langchain.llms import OpenAI

In [9]:
from dotenv import load_dotenv, find_dotenv
# read local .env file with OPENAI_API_KEY [Recommended]
_ = load_dotenv(find_dotenv())

# Direct entry of the api key [NOT recommended to use in production!!!]
# os.environ['OPENAI_API_KEY']="GET your own key from openai.com"

You can get your [OpenAI API key](https://platform.openai.com/account/api-keys) from the OpenAI platform.

# Creating a Langchain Agent and prompts for geospatial analysis

To create a Langchain agent, we’ll use the built-in pandas agent, with the following sequence of prompts to elicit the functions/code snippets required to execute geospatial analysis on the dataset.

In [10]:
import pandas as pd
# Importing the data
df = pd.read_csv('Angel-Hsu-SG-Chicken-Rice-dataset.csv')
df.dropna(inplace=True)
df.head()

,S/N,Stall (Review >10 as of 2023 March 21),Address,Postal code,Operating hour,"Type (Hawker, coffee shop, restaurant)",Google stars,No. of reviews (as of 21 Mar 2023),Cost,Gram of chicken,...,Chilli (Angel's rating),Chilli (Cameraman's rating),Soup (1-10) (Angel's rating),Soup (1-10) (Cameraman's rating),"Saltiness (1-5)\n5 = saltiest, 3 = normal",Rating (10/10) (Angel's rating),Rating (10/10) (Cameraman's rating),"Bacteria (E.coli, Y/N)","E.coli count (CFU/g)\n(>490000 is replaced by 490001, <10 is replaced by 9 for sorting)",Remarks
0,1,Ah Boy Chicken Rice (Mentor Eric Teo),"678A Choa Chu Kang Cres, Singapore 681678",681678,9:30am-8:30pm,Coffee Shop,4.2 stars,303,3.6,60.0,...,9.0,9.0,9.0,9.0,3.0,9.0,9.0,N,9,"Pickled veggie is super nice. Soup is so good,..."
1,2,Ah Five Hainanese Chicken Rice / Fried Rice / ...,"Blk 158 Ang Mo Kio Ave 4, Singapore 560158",560158,8:30am-3:30pm,Coffee Shop,3.9 stars,54,5.5,98.0,...,8.0,9.0,10.0,10.0,3.0,8.5,9.0,N,86,Kampong Chicken was used. Chilli with rice is ...
2,3,Wee Kim Hainanese Chicken Rice,"51 Yishun Ave 11, Singapore 760355",760355,10:30am-8:30pm,Hawker,3.8 stars,13,4.5,68.2,...,8.0,8.5,8.5,8.5,3.0,8.0,8.5,N,59,Beansprout is very crunchy and nice. Sauce is ...
3,4,Henry's Chicken Rice,"117 Commonwealth Dr, Singapore 140117",140117,7am-1pm,Hawker,3.9 stars,73,3.0,98.8,...,4.0,4.0,4.0,4.0,3.0,6.0,6.0,Y,810,"Tofu soup is quite tasteless, only the chicken..."
4,5,Hai Nan Xu Ji Chicken Rice (Stall 144),"84 Marine Parade Central, Singapore 440084",440084,9:30am-2:30pm,Hawker,4.3 stars,15,3.0,77.0,...,5.0,4.0,5.0,6.0,3.0,6.5,6.0,Y,"6,800",Quite cheap and good value. Sauce adds score t...


In [13]:
# Initializing the agent
agent = create_pandas_dataframe_agent(OpenAI(temperature=0),
                                      df, verbose=True)
openai = OpenAI(temperature=0.0)
openai.model_name  # This will print the model being used,
# by default it uses ‘text-davinci-003’

'text-davinci-003'

In [77]:
# Prompt to illicit a function to convert postal code to latitude and longitude

agent("Pls help me develop a function to use  https://developers.onemap.sg/commonapi/ to convert the postal code  in the dataframe to latitude and longitude as new columns. \n Pls use a try-except block to handle any errors when making a request to the OneMap API and return None, None if the postal code is not found.")



> Entering new  chain...
Thought: I need to use the OneMap API to convert the postal code to latitude and longitude
Action: python_repl_ast
Action Input: 
import requests
import json

def get_lat_long(postal_code):
    url = f"https://developers.onemap.sg/commonapi/search?searchVal={postal_code}&returnGeom=Y&getAddrDetails=Y"
    try:
        response = requests.get(url)
        response_json = response.json()
        if response_json['found'] == 0:
            return None, None
        else:
            return response_json['results'][0]['LATITUDE'], response_json['results'][0]['LONGITUDE']
    except:
        return None, None

Observation: 
Thought: I now have a function to convert the postal code to latitude and longitude
Action: python_repl_ast
Action Input: 
df['latitude'], df['longitude'] = zip(*df['Postal code'].apply(get_lat_long))

Observation: 
Thought: I now know the final answer
Final Answer: The function `get_lat_long` can be used to convert the postal code in the dataf

{'input': 'Pls help me develop a function to use  https://developers.onemap.sg/commonapi/ to convert the postal code  in the dataframe to latitude and longitude as new columns. \n Pls use a try-except block to handle any errors when making a request to the OneMap API and return None, None if the postal code is not found.',
 'output': "The function `get_lat_long` can be used to convert the postal code in the dataframe to latitude and longitude as new columns. A try-except block is used to handle any errors when making a request to the OneMap API and return None, None if the postal code is not found. The function can be applied to the dataframe using `df['latitude'], df['longitude'] = zip(*df['Postal code'].apply(get_lat_long))`."}

In [89]:
import requests


def get_lat_long(postal_code):
    url = f"https://developers.onemap.sg/commonapi/search?searchVal={postal_code}&returnGeom=Y&getAddrDetails=Y"
    try:
        response = requests.get(url)
        response_json = response.json()
        if response_json['found'] == 0:
            return None, None
        else:
            return response_json['results'][0]['LATITUDE'], response_json['results'][0]['LONGITUDE']
    except:
        return None, None

In [93]:
# Prompt for using the function get_lat_long on each row of the dataframe df 'Postal code'
agent("Now use the function get_lat_long you developed and a for loop  for index, row in df.iterrows() on each row of the dataframe df 'Postal code' column to calculate values lat, lon for each row of dataframe in the columns 'latitude' and 'longitude' respectively.")



> Entering new  chain...
Thought: I need to use a for loop to iterate through each row of the dataframe
Action: python_repl_ast
Action Input: for index, row in df.iterrows():
Observation: IndentationError: expected an indented block after 'for' statement on line 1 (<unknown>, line 1)
Thought: I need to add the code to the for loop
Action: python_repl_ast
Action Input: for index, row in df.iterrows():
    lat, lon = get_lat_long(row['Postal code'])
    df.loc[index, 'latitude'] = lat
    df.loc[index, 'longitude'] = lon
Observation: 
Thought: I now know the final answer
Final Answer: The for loop should be used to iterate through each row of the dataframe and the get_lat_long function should be used to calculate the values of lat and lon for each row of the dataframe in the columns 'latitude' and 'longitude' respectively.

> Finished chain.


{'input': "Now use the function get_lat_long you developed and a for loop  for index, row in df.iterrows() on each row of the dataframe df 'Postal code' column to calculate values lat, lon for each row of dataframe in the columns 'latitude' and 'longitude' respectively. Execute these loops",
 'output': "The for loop should be used to iterate through each row of the dataframe and the get_lat_long function should be used to calculate the values of lat and lon for each row of the dataframe in the columns 'latitude' and 'longitude' respectively."}

In [95]:
for index, row in df.iterrows():
    lat, lon = get_lat_long(row['Postal code'])
    df.loc[index, 'latitude'] = lat
    df.loc[index, 'longitude'] = lon

df.dropna(inplace=True)

In [99]:
# identifying missing values
agent("How many missing values are there in each column?")



> Entering new  chain...
Thought: I need to find out how many missing values are in each column
Action: python_repl_ast
Action Input: df.isnull().sum()
Observation: S/N                                          0
Stall                                        0
Address                                      0
Postal code                                  0
Operating hour                               0
Type (Hawker, coffee shop, restaurant)       0
Google stars                                 0
No. of reviews (as of 21 Mar 2023)           0
Cost                                         0
Gram of chicken                              0
Cost per gram                                0
Tenderness (Angel's rating)                  0
Tenderness (Cameraman's rating)              0
Rice Fragrance (Angel's rating)              0
Rice Fragrance (Cameraman's rating)          0
Chilli (Angel's rating)                      0
Chilli (Cameraman's rating)                  0
Soup (1-10) (Angel's rating)      

{'input': 'How many missing values are there in each column?',
 'output': 'There are no missing values in any of the columns.'}

In [109]:
# Prompt for using the folium library to plot SG map with popups, tooltips
agent(
    "Pls use folium library to plot a map of Singapore with markers for each row of the dataframe df. The markers should have a popup with the text 'Coordinates: {latitude}, {longitude}' where latitude and longitude are the values in the columns 'latitude' and 'longitude' respectively. \n Pls also add tooltips for each row for all the columns in the dataframe df using iterrows. \n For each column in the dataframe df, the tooltip should have the text '{column_name}: {column_value}', followed by a line break. Use folium.Tooltip(tooltip_text) to create the tooltip and marker.add_child(tooltip) to add the tooltip to the marker.")



> Entering new  chain...
Thought: I need to use folium library to plot a map of Singapore with markers for each row of the dataframe df.
Action: python_repl_ast
Action Input: import folium
Observation: 
Thought: I need to get the coordinates of each row in the dataframe df.
Action: python_repl_ast
Action Input: df['latitude'], df['longitude']
Observation: (0     1.40194926829714
1      1.3747805874721
2     1.42540416849826
3     1.30659245003577
4     1.30228520968842
            ...       
95    1.31703502375324
96    1.32416941474156
97    1.30717070558181
98    1.33171102182223
99    1.30206450556085
Name: latitude, Length: 93, dtype: object, 0     103.744856521894
1     103.838997571585
2     103.844843123768
3     103.800270024799
4     103.906288669909
            ...       
95    103.879309854774
96    103.851769354283
97    103.906377723732
98    103.923951085318
99    103.836340953476
Name: longitude, Length: 93, dtype: object)
Thought: I need to create a map of Singapore a

{'input': "Pls use folium library to plot a map of Singapore with markers for each row of the dataframe df. The markers should have a popup with the text 'Coordinates: {latitude}, {longitude}' where latitude and longitude are the values in the columns 'latitude' and 'longitude' respectively. \n Pls also add tooltips for each row for all the columns in the dataframe df using iterrows. \n For each column in the dataframe df, the tooltip should have the text '{column_name}: {column_value}', followed by a line break. Use folium.Tooltip(tooltip_text) to create the tooltip and marker.add_child(tooltip) to add the tooltip to the marker.",
 'output': "The final answer is to use the folium library to plot a map of Singapore with markers for each row of the dataframe df. The markers should have a popup with the text 'Coordinates: {latitude}, {longitude}' where latitude and longitude are the values in the columns 'latitude' and 'longitude' respectively. Pls also add tooltips for each row for all 

We have also added lines of code in the following cells to color code the markers on the map based on the E.coli levels of the chicken rice stalls, so that the user can easily view at a glance which chicken rice stalls have acceptable E.coli levels and which do not.

For 
- acceptable E.coli levels (<100), the marker will be green.
- E.coli levels between 100-1000, the marker will be orange.
- E.coli levels >1000, the marker will be red.

The user can hover over any marker to view the name of the chicken rice stall, its food rating, cost, and E.coli level, and other information in its tooltip, to make an informed decision on which chicken rice stall to visit.

The abridged version of the marker tooltip shall be given in the Appendix, with only the fields name of the chicken rice stall, its food rating, cost, and E.coli level displayed in the marker tooltip.

In [33]:
import folium

m = folium.Map(location=[1.3521, 103.8198], zoom_start=11)

for index, row in df.iterrows():
    # Determine the color of the marker based on E.coli count
    if row['E.coli count (CFU/g)'] < 100:
        marker_color = 'green'
    elif 100 <= row['E.coli count (CFU/g)'] < 1000:
        marker_color = 'orange'
    else:
        marker_color = 'red'
    marker = folium.Marker(location=[row['latitude'], row['longitude']],
                           popup='Coordinates: {}, {}'.format(
                               row['latitude'], row['longitude']),
                           icon=folium.Icon(color=marker_color))
    m.add_child(marker)
    tooltip_text = ''
    for col in df.columns:
        tooltip_text += '{}: {}<br>'.format(col, row[col])
    tooltip = folium.Tooltip(tooltip_text)
    marker.add_child(tooltip)
m

Please view the Appendix below for sample code generated by Langchain, and modified so that selected columns are only displayed on the tooltip.

# Appendix [Scripts without using Langchain]

In [14]:
import pandas as pd
# Importing the data
df = pd.read_csv('Angel-Hsu-SG-Chicken-Rice-dataset.csv')
df.dropna(inplace=True)

df.rename(columns={
          'E.coli count (CFU/g)\n(>490000 is replaced by 490001, <10 is replaced by 9 for sorting)': 'E.coli count (CFU/g)'}, inplace=True)
df.rename(columns={'Rating (10/10) (Angel\'s rating)': 'Rating'}, inplace=True)
df.rename(
    columns={'Stall (Review >10 as of 2023 March 21)': 'Stall'}, inplace=True)

In [15]:
import requests


def get_geocoordinates(postal_code):
    url = f"https://developers.onemap.sg/commonapi/search?searchVal={postal_code}&returnGeom=Y&getAddrDetails=N"
    response = requests.get(url)
    data = response.json()

    if 'results' in data and len(data['results']) > 0:
        latitude = data['results'][0]['LATITUDE']
        longitude = data['results'][0]['LONGITUDE']
        return latitude, longitude
    else:
        return None, None

In [16]:
df['latitude'] = None
df['longitude'] = None

for index, row in df.iterrows():
    postal_code = row['Postal code']
    latitude, longitude = get_geocoordinates(postal_code)
    df.at[index, 'latitude'] = latitude
    df.at[index, 'longitude'] = longitude

df.dropna(inplace=True)

In [28]:
df['E.coli count (CFU/g)'] = df['E.coli count (CFU/g)'].str.replace(',',
                                                                    '').astype(int)

In [34]:
import folium
singapore_map = folium.Map(location=[1.3521, 103.8198], zoom_start=12)

for index, row in df.iterrows():
    # Determine the color of the marker based on E.coli count
    if row['E.coli count (CFU/g)'] < 100:
        marker_color = 'green'
    elif 100 <= row['E.coli count (CFU/g)'] < 1000:
        marker_color = 'orange'
    else:
        marker_color = 'red'

    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Coordinates: {row['latitude']}, {row['longitude']}",
        icon=folium.Icon(color=marker_color)  # Use the determined color
    )

    tooltip_text = f"E.coli count (CFU/g): {row['E.coli count (CFU/g)']}<br>Cost: {row['Cost']}<br>Rating: {row['Rating']}<br>Stall: {row['Stall']}<br>Address: {row['Address']}"
    tooltip = folium.Tooltip(tooltip_text)
    marker.add_child(tooltip)
    marker.add_to(singapore_map)

singapore_map